# Download Data

In [7]:
import xgboost as xgb
import os
import snowflake.connector


con = snowflake.connector.connect(
    user=os.environ['SFUSER'],
    password=os.environ['SFPASSWORD'],
    account='SUMMITDEMO2',
    warehouse='LARGE',
    database='ML',
    schema='demo',
    role='ACCOUNTADMIN'
)

results = con.cursor().execute("select libsvm from chicago_taxi_libsvm sample (20000 rows);").fetchall()
with open('./chicago_taxi_training.libsvm', 'w') as f:
    for l in results:
        f.write(l[0] + '\n')
        

# Train Models

In [8]:
dtrain = xgb.DMatrix('./chicago_taxi_training.libsvm')

params = {'max_depth': 3, 'eta': 0.1, 'objective': 'reg:linear'}
print('model1: %s' % params)
taxi_model = xgb.train(params, dtrain, evals=[(dtrain, 'train')] , num_boost_round=20)
taxi_model.dump_model('./chicago_3_20.json', dump_format='json')

params = {'max_depth': 4, 'eta': 0.1, 'objective': 'reg:linear'}
print('model2: %s' % params)
taxi_model = xgb.train(params, dtrain, evals=[(dtrain, 'train')] , num_boost_round=20)
taxi_model.dump_model('./chicago_4_20.json', dump_format='json')

params = {'max_depth': 4, 'eta': 0.1, 'objective': 'reg:linear'}
print('model3: %s' % params)
taxi_model = xgb.train(params, dtrain, evals=[(dtrain, 'train')] , num_boost_round=40)
taxi_model.dump_model('./chicago_4_40.json', dump_format='json')


[14:42:20] 20000x315 matrix with 180000 entries loaded from ./chicago_taxi_training.libsvm
model1: {'max_depth': 3, 'eta': 0.1, 'objective': 'reg:linear'}
[0]	train-rmse:13.2638
[1]	train-rmse:12.1464
[2]	train-rmse:11.1579
[3]	train-rmse:10.2864
[4]	train-rmse:9.52052
[5]	train-rmse:8.85039
[6]	train-rmse:8.26558
[7]	train-rmse:7.75728
[8]	train-rmse:7.31808
[9]	train-rmse:6.93935
[10]	train-rmse:6.61491
[11]	train-rmse:6.33328
[12]	train-rmse:6.09544
[13]	train-rmse:5.89443
[14]	train-rmse:5.72464
[15]	train-rmse:5.58108
[16]	train-rmse:5.46011
[17]	train-rmse:5.28108
[18]	train-rmse:5.13026
[19]	train-rmse:4.96871
model2: {'max_depth': 4, 'eta': 0.1, 'objective': 'reg:linear'}
[0]	train-rmse:13.2394
[1]	train-rmse:12.1022
[2]	train-rmse:11.0921
[3]	train-rmse:10.198
[4]	train-rmse:9.40884
[5]	train-rmse:8.70838
[6]	train-rmse:8.09812
[7]	train-rmse:7.55605
[8]	train-rmse:7.07783
[9]	train-rmse:6.67289
[10]	train-rmse:6.31651
[11]	train-rmse:5.99769
[12]	train-rmse:5.74041
[13]	train

In [12]:
!cat ./chicago_3_20.json

[
  { "nodeid": 0, "depth": 0, "split": "f1", "split_condition": -0.107625604, "yes": 1, "no": 2, "missing": 1, "children": [
    { "nodeid": 1, "depth": 1, "split": "f3", "split_condition": 0.114826955, "yes": 3, "no": 4, "missing": 3, "children": [
      { "nodeid": 3, "depth": 2, "split": "f142", "split_condition": -9.53674316e-07, "yes": 7, "no": 8, "missing": 7, "children": [
        { "nodeid": 7, "leaf": 1.32088125 },
        { "nodeid": 8, "leaf": 3.00807691 }
      ]},
      { "nodeid": 4, "depth": 2, "split": "f1", "split_condition": -0.691340327, "yes": 9, "no": 10, "missing": 9, "children": [
        { "nodeid": 9, "leaf": 3.73282242 },
        { "nodeid": 10, "leaf": 2.91024923 }
      ]}
    ]},
    { "nodeid": 2, "depth": 1, "split": "f3", "split_condition": -0.117497236, "yes": 5, "no": 6, "missing": 5, "children": [
      { "nodeid": 5, "depth": 2, "split": "f3", "split_condition": -0.624133646, "yes": 11, "no": 12, "missing": 11, "children": [
        { "nodeid": 11, 

# Upload Models

In [9]:
con.cursor().execute("create or replace table chicago_taxi_model (name string, model variant);");

In [11]:
def upload_model(file_name, con):
    with open(file_name, 'r') as file:
        content = file.read()
    
    sql = """insert into chicago_taxi_model
    select column1 as name, parse_json(column2) as model
    from values ('%s', '%s') as vals;""" % (file_name, content)
    con.cursor().execute(sql)
    

upload_model('chicago_3_20.json', con)
upload_model('chicago_4_20.json', con)
upload_model('chicago_4_40.json', con)